In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1575675000085_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [2]:
# Read in business.json
business = sc.textFile("s3a://dsfinal/business.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import json

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# JSON to dataframe
df_business = spark.read.json(business)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Save all restaurants into a dataframe
df_restaurant_only = df_business.filter(df_business.categories.contains('Restaurants'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Restaurants with more than 200 reviews
popular_restaurants = df_restaurant_only.filter(df_restaurant_only['review_count'] > 200)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Read in review.json
reviews = sc.textFile("s3a://dsfinal/review.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Put reviews in dataframe
df_reviews = spark.read.json(reviews)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Save list of restaurant ids with > 200 reviews
restaurant_ids_list = popular_restaurants.select('business_id').rdd.flatMap(lambda x: x).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# restaurant_reviews has reviews of restaurants with > 200 reviews
restaurant_reviews = df_reviews.where(df_reviews.business_id.isin(restaurant_ids_list))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Only have reviews with at least one useful vote
restaurant_reviews_useful = restaurant_reviews.filter(restaurant_reviews['useful'] > 0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df5 = restaurant_reviews_useful.filter(restaurant_reviews['stars'] == 5.0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df4 = restaurant_reviews_useful.filter(restaurant_reviews['stars'] == 4.0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df3 = restaurant_reviews_useful.filter(restaurant_reviews['stars'] == 3.0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df2 = restaurant_reviews_useful.filter(restaurant_reviews['stars'] == 2.0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df1 = restaurant_reviews_useful.filter(restaurant_reviews['stars'] == 1.0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Balance the data

In [17]:
sampleRatio5 = 91988/338840
undersampled_5 = df5.sample(False, sampleRatio5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
sampleRatio4 = 91988/243028
undersampled_4 = df4.sample(False, sampleRatio4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
sampleRatio3 = 91988/121834
undersampled_3 = df3.sample(False, sampleRatio3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
sampleRatio1 = 91988/101465
undersampled_1 = df1.sample(False, sampleRatio1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
total1 = undersampled_1.unionAll(undersampled_3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
total2 = total1.unionAll(undersampled_4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
total3 = total2.unionAll(undersampled_5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
balanced_review_data = total3.unionAll(df2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Only save important columns of stars and text
balanced_review_data = balanced_review_data.select("stars", "text")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
balanced_review_data.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+
|stars|                text|
+-----+--------------------+
|  1.0|We had dinner at ...|
|  1.0|I am so blown awa...|
|  1.0|Worse than before...|
+-----+--------------------+
only showing top 3 rows

In [27]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, NGram
from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
# stop words
#add_stopwords = StopWordsRemover.loadDefaultStopWords("english")
#stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
# ngram
ngram = NGram(n=2, inputCol="words", outputCol="ngrams")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml import Pipeline
hashingTF = HashingTF(inputCol="ngrams", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=30) #minDocFreq: remove sparse terms
pipeline = Pipeline(stages=[regexTokenizer, ngram, hashingTF, idf])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(balanced_review_data)
dataset = pipelineFit.transform(balanced_review_data)
dataset.show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [30]:
# Split dataset into training and testing sets
(trainingData, testData) = dataset.randomSplit([0.8, 0.2], seed = 100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
#lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial", labelCol='stars', featuresCol='features')
#lrModel = lr.fit(trainingData)
#predictions = lrModel.transform(testData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# Naive Bayes model
# Fitting model to training set
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1, labelCol='stars', featuresCol='features')
model = nb.fit(trainingData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
# Test model on testing set
predictions = model.transform(testData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
predictions.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|stars|                text|               words|              ngrams|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  1.0|15 minute wait fo...|[15, minute, wait...|[15 minute, minut...|(10000,[28,185,29...|(10000,[28,185,29...|[-3810.3983073612...|[0.00375414964833...|       1.0|
|  1.0|A group of us wen...|[a, group, of, us...|[a group, group o...|(10000,[246,529,5...|(10000,[246,529,5...|[-3050.2853238468...|[0.99921142583358...|       0.0|
|  1.0|A very pricey $50...|[a, very, pricey,...|[a very, very pri...|(10000,[97,303,52...|(10000,[97,303,52...|[-2087.1122353244...|[0.00110473079859...|       1.0|
+---

In [42]:
# Model predicts classes from 0.0 to 4.0, so create new column named adj_preds with value prediction + 1
right_preds = predictions.withColumn('adj_preds', predictions.prediction + 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
right_preds.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+---------+
|stars|                text|               words|              ngrams|         rawFeatures|            features|label|       rawPrediction|         probability|prediction|adj_preds|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+---------+
|  1.0|15 minute wait fo...|[15, minute, wait...|[15 minute, minut...|(10000,[28,185,29...|(10000,[28,185,29...|  2.0|[-3809.8683497373...|[0.00929929764748...|       1.0|      2.0|
|  1.0|A coworker and I ...|[a, coworker, and...|[a coworker, cowo...|(10000,[27,91,97,...|(10000,[27,91,97,...|  2.0|[-9802.0964860878...|[0.08693779318946...|       1.0|      2.0|
+-----+--------------------+--------------------+--------------------+--------------------

In [43]:
# Calculate accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="adj_preds", labelCol="stars", metricName="accuracy")
evaluator.evaluate(right_preds)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5318363109872698

In [49]:
# Calculate F1 score
evaluator = MulticlassClassificationEvaluator(predictionCol="adj_preds", labelCol="stars", metricName="f1")
evaluator.evaluate(right_preds)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.526830735691193

In [44]:
# Save labels and predictions into separate rdds
rddLabels = predictions.rdd.map(lambda x:x.stars)
rddPreds = predictions.rdd.map(lambda x:x.prediction + 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
# Combine predictions and labels into one rdd
preds_and_labels = rddPreds.zip(rddLabels).map(
    lambda x: [x[0], x[1]])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
# Output confusion matrix 
from pyspark.mllib.evaluation import MulticlassMetrics

metrics = MulticlassMetrics(preds_and_labels)

metrics.confusionMatrix().toArray()
## array([[ 2.,  0.],
##        [ 0.,  2.]])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[11822.,  4215.,  1133.,   496.,   749.],
       [ 4366.,  7148.,  4722.,  1184.,   797.],
       [ 1418.,  3164.,  8564.,  3944.,  1270.],
       [  537.,   862.,  3055.,  8387.,  5404.],
       [  538.,   439.,   782.,  3769., 12750.]])

In [31]:
# Random Forest Model
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="stars", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)
# Train model with Training Data
rfModel = rf.fit(trainingData)
predictionsrf = rfModel.transform(testData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
predictionsrf.select("stars", "prediction").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------+
|stars|prediction|
+-----+----------+
|  1.0|       3.0|
|  1.0|       2.0|
|  1.0|       2.0|
|  1.0|       5.0|
|  1.0|       5.0|
|  1.0|       5.0|
|  1.0|       1.0|
|  1.0|       5.0|
|  1.0|       5.0|
|  1.0|       5.0|
+-----+----------+
only showing top 10 rows

In [34]:
rddLabelsRF = predictionsrf.rdd.map(lambda x:x.stars)
rddPredsRF = predictionsrf.rdd.map(lambda x:x.prediction)
# Combine predictions and labels into one rdd
preds_and_labelsRF = rddPredsRF.zip(rddLabelsRF).map(
    lambda x: [x[0], x[1]])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
from pyspark.mllib.evaluation import MulticlassMetrics

metrics = MulticlassMetrics(preds_and_labelsRF)

metrics.confusionMatrix().toArray()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[ 9565.,  2200.,   851.,   406.,  5439.],
       [ 4750.,  4168.,  3076.,  1019.,  5204.],
       [ 2069.,  2402.,  6131.,  2171.,  5591.],
       [ 1142.,  1083.,  3411.,  3741.,  8935.],
       [ 1015.,   657.,  1358.,  2152., 13118.]])

In [37]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="stars", metricName="f1")
evaluator.evaluate(predictionsrf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.38217367102697736

In [ ]:
# Support Vector Machine
from pyspark.mllib.classification import SVMWithSGD
svm = SVMWithSGD()

In [71]:
# Logistic Regression
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial", labelCol='stars', featuresCol='features')
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)

VBox()

An error was encountered:
Invalid status code '404' from https://172.31.23.248:18888/sessions/0 with error payload: {"msg":"Session '0' not found."}
